In [4]:
@everywhere push!(LOAD_PATH, "C:\\Users\\LinFan Xiao\\Academics\\College\\Capstone\\work_in_julia")
include("./construct_adjoint.jl")
include("./unit_tests.jl")
using construct_adjoint
using unit_tests

In [5]:
t = symbols("t")

t

In [7]:
symL = SymLinearDifferentialOperator([t+1 t+1 t+1], (0,1), t)

SymLinearDifferentialOperator(SymPy.Sym[t + 1 t + 1 t + 1], (0, 1), t)

In [8]:
L = LinearDifferentialOperator([t->t+1, t->t+1, t->t+1], (0,1), symL)

LinearDifferentialOperator(Function[#243, #244, #245], (0, 1), SymLinearDifferentialOperator(SymPy.Sym[t + 1 t + 1 t + 1], (0, 1), t))

In [9]:
u, v = SymFunction("u")(t), SymFunction("v")(t)

(u(t), v(t))

In [10]:
symUVForm = get_symUvForm(L, u, v; substitute = true)
pprint(symUVForm)

       ____          d /____\     ____ d               d /____\   ____ d      
t*u(t)*v(t) - t*u(t)*--\v(t)/ + t*v(t)*--(u(t)) - u(t)*--\v(t)/ + v(t)*--(u(t)
                     dt                dt              dt              dt     

 
)
 


In [11]:
symUVForm = get_symUvForm(L, u, v; substitute = false)
pprint(symUVForm)

             d /____\         ____ d                     ____        ____ d   
- p0(t)*u(t)*--\v(t)/ + p0(t)*v(t)*--(u(t)) + p1(t)*u(t)*v(t) - u(t)*v(t)*--(p
             dt                    dt                                     dt  

     
0(t))
     


In [12]:
symDerivMatrix = get_symPDerivMatrix(L; substitute = true)

2×2 Array{SymPy.Sym,2}:
 t + 1  1
 t + 1  1

In [10]:
symDerivMatrix = get_symPDerivMatrix(L; substitute = false)

2×2 Array{SymPy.Sym,2}:
 p0(t)  Derivative(p0(t), t)
 p1(t)  Derivative(p1(t), t)

In [13]:
pDerivMatrix = [t->t+1 t->t; t->t+1 t->t]

2×2 Array{Function,2}:
 #249  #250
 #251  #252

In [20]:
B = get_B(L; pDerivMatrix = pDerivMatrix)
# B[2,1](1)

-2

In [13]:
symB = get_B(L; symbolic = true)

2×2 Array{Union{Function, Number},2}:
      t  t + 1
 -t - 1      0

In [15]:
symB = get_B(L; symbolic = true, substitute = false)
pprint(symB[1])

        d        
p1(t) - --(p0(t))
        dt       


In [14]:
BHat = get_BHat(L, B)

4×4 Array{Number,2}:
 -1  -1   0  0
  1   0   0  0
  0   0   1  2
  0   0  -2  0

In [15]:
symXi = get_symXi(L; substitute = true, xDef = t^2+2)

2×1 Array{SymPy.Sym,2}:
 t^2 + 2
     2*t

In [16]:
evaluate_xi(L, 1, symXi)

2×1 Array{Number,2}:
 3
 2

In [17]:
xi = [t->t^2+2; t->2t]

2-element Array{Function,1}:
 #77
 #78

In [18]:
evaluate_xi(L, 1, xi)

2×1 Array{Number,2}:
 3
 2

In [19]:
U = VectorBoundaryForm([1 2; 3 4], [4 3; 2 1])
get_boundaryCondition(L, U, symXi)

2×1 Array{SymPy.Sym,2}:
 20
 14

In [20]:
get_boundaryCondition(L, U, xi)

2×1 Array{Int64,2}:
 20
 14

In [21]:
for n = 1:10
    println(test_generate_adjoint(n, 10))
end

Test 1
Testing the algorithm to generate valid adjoint U+: Constant p_k
Testing: order of L = 1
Passed!
Testing the algorithm to generate valid adjoint U+: Variable p_k
Testing: order of L = 1
Passed!
Testing the algorithm to generate valid adjoint U+: Constant or variable p_k
Testing: order of L = 1
Passed!
Test 2
Testing the algorithm to generate valid adjoint U+: Constant p_k
Testing: order of L = 1
Passed!
Testing the algorithm to generate valid adjoint U+: Variable p_k
Testing: order of L = 1
Passed!
Testing the algorithm to generate valid adjoint U+: Constant or variable p_k
Testing: order of L = 1
Passed!
Test 3
Testing the algorithm to generate valid adjoint U+: Constant p_k
Testing: order of L = 1
Passed!
Testing the algorithm to generate valid adjoint U+: Variable p_k
Testing: order of L = 1
Passed!
Testing the algorithm to generate valid adjoint U+: Constant or variable p_k
Testing: order of L = 1
Passed!
Test 4
Testing the algorithm to generate valid adjoint U+: Constant p_

Testing: order of L = 3
Passed!
Testing the algorithm to generate valid adjoint U+: Constant or variable p_k
Testing: order of L = 3
Passed!
Test 8
Testing the algorithm to generate valid adjoint U+: Constant p_k
Testing: order of L = 3
Passed!
Testing the algorithm to generate valid adjoint U+: Variable p_k
Testing: order of L = 3
Passed!
Testing the algorithm to generate valid adjoint U+: Constant or variable p_k
Testing: order of L = 3
Passed!
Test 9
Testing the algorithm to generate valid adjoint U+: Constant p_k
Testing: order of L = 3
Passed!
Testing the algorithm to generate valid adjoint U+: Variable p_k
Testing: order of L = 3
Passed!
Testing the algorithm to generate valid adjoint U+: Constant or variable p_k
Testing: order of L = 3
Passed!
Test 10
Testing the algorithm to generate valid adjoint U+: Constant p_k
Testing: order of L = 3
Passed!
Testing the algorithm to generate valid adjoint U+: Variable p_k
Testing: order of L = 3
Passed!
Testing the algorithm to generate val

Testing: order of L = 6
Passed!
Testing the algorithm to generate valid adjoint U+: Constant or variable p_k
Testing: order of L = 6
Passed!
Test 5
Testing the algorithm to generate valid adjoint U+: Constant p_k
Testing: order of L = 6
Passed!
Testing the algorithm to generate valid adjoint U+: Variable p_k
Testing: order of L = 6
Passed!
Testing the algorithm to generate valid adjoint U+: Constant or variable p_k
Testing: order of L = 6
Passed!
Test 6
Testing the algorithm to generate valid adjoint U+: Constant p_k
Testing: order of L = 6
Passed!
Testing the algorithm to generate valid adjoint U+: Variable p_k
Testing: order of L = 6
Passed!
Testing the algorithm to generate valid adjoint U+: Constant or variable p_k
Testing: order of L = 6
Passed!
Test 7
Testing the algorithm to generate valid adjoint U+: Constant p_k
Testing: order of L = 6
Passed!
Testing the algorithm to generate valid adjoint U+: Variable p_k
Testing: order of L = 6
Passed!
Testing the algorithm to generate vali

Passed!
true
Test 1
Testing the algorithm to generate valid adjoint U+: Constant p_k
Testing: order of L = 9
Passed!
Testing the algorithm to generate valid adjoint U+: Variable p_k
Testing: order of L = 9
Passed!
Testing the algorithm to generate valid adjoint U+: Constant or variable p_k
Testing: order of L = 9
Passed!
Test 2
Testing the algorithm to generate valid adjoint U+: Constant p_k
Testing: order of L = 9
Passed!
Testing the algorithm to generate valid adjoint U+: Variable p_k
Testing: order of L = 9
Passed!
Testing the algorithm to generate valid adjoint U+: Constant or variable p_k
Testing: order of L = 9
Passed!
Test 3
Testing the algorithm to generate valid adjoint U+: Constant p_k
Testing: order of L = 9
Passed!
Testing the algorithm to generate valid adjoint U+: Variable p_k
Testing: order of L = 9
Passed!
Testing the algorithm to generate valid adjoint U+: Constant or variable p_k
Testing: order of L = 9
Passed!
Test 4
Testing the algorithm to generate valid adjoint U+

In [22]:
for n = 1:10
    println(test_symLinearDifferentialOperatorDef(n, 10))
end

Test 1
Testing definition of SymLinearDifferentialOperator: symP_k are SymPy.Sym
Passed!
Testing definition of SymLinearDifferentialOperator: symP_k are Number
Passed!
Testing definition of SymLinearDifferentialOperator: symP_k are SymPy.Sym and Number
Passed!
Testing StructDefinitionError: symP_k should be SymPy.Sym or Number
Passed!
Testing StructDefinitionError: Only one free symbol is allowed in symP_k
Passed!
Test 2
Testing definition of SymLinearDifferentialOperator: symP_k are SymPy.Sym
Passed!
Testing definition of SymLinearDifferentialOperator: symP_k are Number
Passed!
Testing definition of SymLinearDifferentialOperator: symP_k are SymPy.Sym and Number
Passed!
Testing StructDefinitionError: symP_k should be SymPy.Sym or Number
Passed!
Testing StructDefinitionError: Only one free symbol is allowed in symP_k
Passed!
Test 3
Testing definition of SymLinearDifferentialOperator: symP_k are SymPy.Sym
Passed!
Testing definition of SymLinearDifferentialOperator: symP_k are Number
Pass

Testing definition of SymLinearDifferentialOperator: symP_k are Number
Passed!
Testing definition of SymLinearDifferentialOperator: symP_k are SymPy.Sym and Number
Passed!
Testing StructDefinitionError: symP_k should be SymPy.Sym or Number
Passed!
Testing StructDefinitionError: Only one free symbol is allowed in symP_k
Passed!
Test 7
Testing definition of SymLinearDifferentialOperator: symP_k are SymPy.Sym
Passed!
Testing definition of SymLinearDifferentialOperator: symP_k are Number
Passed!
Testing definition of SymLinearDifferentialOperator: symP_k are SymPy.Sym and Number
Passed!
Testing StructDefinitionError: symP_k should be SymPy.Sym or Number
Passed!
Testing StructDefinitionError: Only one free symbol is allowed in symP_k
Passed!
Test 8
Testing definition of SymLinearDifferentialOperator: symP_k are SymPy.Sym
Passed!
Testing definition of SymLinearDifferentialOperator: symP_k are Number
Passed!
Testing definition of SymLinearDifferentialOperator: symP_k are SymPy.Sym and Number


Passed!
Testing StructDefinitionError: Only one free symbol is allowed in symP_k
Passed!
Test 6
Testing definition of SymLinearDifferentialOperator: symP_k are SymPy.Sym
Passed!
Testing definition of SymLinearDifferentialOperator: symP_k are Number
Passed!
Testing definition of SymLinearDifferentialOperator: symP_k are SymPy.Sym and Number
Passed!
Testing StructDefinitionError: symP_k should be SymPy.Sym or Number
Passed!
Testing StructDefinitionError: Only one free symbol is allowed in symP_k
Passed!
Test 7
Testing definition of SymLinearDifferentialOperator: symP_k are SymPy.Sym
Passed!
Testing definition of SymLinearDifferentialOperator: symP_k are Number
Passed!
Testing definition of SymLinearDifferentialOperator: symP_k are SymPy.Sym and Number
Passed!
Testing StructDefinitionError: symP_k should be SymPy.Sym or Number
Passed!
Testing StructDefinitionError: Only one free symbol is allowed in symP_k
Passed!
Test 8
Testing definition of SymLinearDifferentialOperator: symP_k are SymP

Passed!
Testing definition of SymLinearDifferentialOperator: symP_k are Number
Passed!
Testing definition of SymLinearDifferentialOperator: symP_k are SymPy.Sym and Number
Passed!
Testing StructDefinitionError: symP_k should be SymPy.Sym or Number
Passed!
Testing StructDefinitionError: Only one free symbol is allowed in symP_k
Passed!
Test 9
Testing definition of SymLinearDifferentialOperator: symP_k are SymPy.Sym
Passed!
Testing definition of SymLinearDifferentialOperator: symP_k are Number
Passed!
Testing definition of SymLinearDifferentialOperator: symP_k are SymPy.Sym and Number
Passed!
Testing StructDefinitionError: symP_k should be SymPy.Sym or Number
Passed!
Testing StructDefinitionError: Only one free symbol is allowed in symP_k
Passed!
Test 10
Testing definition of SymLinearDifferentialOperator: symP_k are SymPy.Sym
Passed!
Testing definition of SymLinearDifferentialOperator: symP_k are Number
Passed!
Testing definition of SymLinearDifferentialOperator: symP_k are SymPy.Sym an

Passed!
Testing definition of SymLinearDifferentialOperator: symP_k are Number
Passed!
Testing definition of SymLinearDifferentialOperator: symP_k are SymPy.Sym and Number
Passed!
Testing StructDefinitionError: symP_k should be SymPy.Sym or Number
Passed!
Testing StructDefinitionError: Only one free symbol is allowed in symP_k
Passed!
Test 10
Testing definition of SymLinearDifferentialOperator: symP_k are SymPy.Sym
Passed!
Testing definition of SymLinearDifferentialOperator: symP_k are Number
Passed!
Testing definition of SymLinearDifferentialOperator: symP_k are SymPy.Sym and Number
Passed!
Testing StructDefinitionError: symP_k should be SymPy.Sym or Number
Passed!
Testing StructDefinitionError: Only one free symbol is allowed in symP_k
Passed!
true
Test 1
Testing definition of SymLinearDifferentialOperator: symP_k are SymPy.Sym
Passed!
Testing definition of SymLinearDifferentialOperator: symP_k are Number
Passed!
Testing definition of SymLinearDifferentialOperator: symP_k are SymPy.S

In [23]:
for n = 1:10
    println(test_linearDifferentialOperatorDef(n, 10))
end

Test 10
Testing definition of LinearDifferentialOperator: p_k are variable Function
Passed!
Testing definition of LinearDifferentialOperator: p_k are Constants
Passed!
Testing definition of LinearDifferentialOperator: p_k are mixed
Passed!
Testing StructDefinitionError: p_k should be Function or Number
Passed!
Testing StructDefinitionError: Number of p_k and symP_k do not match
Passed!
Testing StructDefinitionError: Intervals of L and symL do not match
Passed!
Testing StructDefinitionError: p0 vanishes on [a,b]
Passed!
Test 10
Testing definition of LinearDifferentialOperator: p_k are variable Function
Passed!
Testing definition of LinearDifferentialOperator: p_k are Constants
Passed!
Testing definition of LinearDifferentialOperator: p_k are mixed
Passed!
Testing StructDefinitionError: p_k should be Function or Number
Passed!
Testing StructDefinitionError: Number of p_k and symP_k do not match
Passed!
Testing StructDefinitionError: Intervals of L and symL do not match
Passed!
Testing St

Passed!
Testing definition of LinearDifferentialOperator: p_k are Constants
Passed!
Testing definition of LinearDifferentialOperator: p_k are mixed
Passed!
Testing StructDefinitionError: p_k should be Function or Number
Passed!
Testing StructDefinitionError: Number of p_k and symP_k do not match
Passed!
Testing StructDefinitionError: Intervals of L and symL do not match
Passed!
Testing StructDefinitionError: p0 vanishes on [a,b]
Passed!
Test 10
Testing definition of LinearDifferentialOperator: p_k are variable Function
Passed!
Testing definition of LinearDifferentialOperator: p_k are Constants
Passed!
Testing definition of LinearDifferentialOperator: p_k are mixed
Passed!
Testing StructDefinitionError: p_k should be Function or Number
Passed!
Testing StructDefinitionError: Number of p_k and symP_k do not match
Passed!
Testing StructDefinitionError: Intervals of L and symL do not match
Passed!
Testing StructDefinitionError: p0 vanishes on [a,b]
Passed!
Test 10
Testing definition of Line

Passed!
Testing definition of LinearDifferentialOperator: p_k are Constants
Passed!
Testing definition of LinearDifferentialOperator: p_k are mixed
Passed!
Testing StructDefinitionError: p_k should be Function or Number
Passed!
Testing StructDefinitionError: Number of p_k and symP_k do not match
Passed!
Testing StructDefinitionError: Intervals of L and symL do not match
Passed!
Testing StructDefinitionError: p0 vanishes on [a,b]
Passed!
Test 10
Testing definition of LinearDifferentialOperator: p_k are variable Function
Passed!
Testing definition of LinearDifferentialOperator: p_k are Constants
Passed!
Testing definition of LinearDifferentialOperator: p_k are mixed
Passed!
Testing StructDefinitionError: p_k should be Function or Number
Passed!
Testing StructDefinitionError: Number of p_k and symP_k do not match
Passed!
Testing StructDefinitionError: Intervals of L and symL do not match
Passed!
Testing StructDefinitionError: p0 vanishes on [a,b]
Passed!
Test 10
Testing definition of Line

Passed!
Testing definition of LinearDifferentialOperator: p_k are Constants
Passed!
Testing definition of LinearDifferentialOperator: p_k are mixed
Passed!
Testing StructDefinitionError: p_k should be Function or Number
Passed!
Testing StructDefinitionError: Number of p_k and symP_k do not match
Passed!
Testing StructDefinitionError: Intervals of L and symL do not match
Passed!
Testing StructDefinitionError: p0 vanishes on [a,b]
Passed!
Test 10
Testing definition of LinearDifferentialOperator: p_k are variable Function
Passed!
Testing definition of LinearDifferentialOperator: p_k are Constants
Passed!
Testing definition of LinearDifferentialOperator: p_k are mixed
Passed!
Testing StructDefinitionError: p_k should be Function or Number
Passed!
Testing StructDefinitionError: Number of p_k and symP_k do not match
Passed!
Testing StructDefinitionError: Intervals of L and symL do not match
Passed!
Testing StructDefinitionError: p0 vanishes on [a,b]
Passed!
true
Test 10
Testing definition of

Passed!
Testing definition of LinearDifferentialOperator: p_k are Constants
Passed!
Testing definition of LinearDifferentialOperator: p_k are mixed
Passed!
Testing StructDefinitionError: p_k should be Function or Number
Passed!
Testing StructDefinitionError: Number of p_k and symP_k do not match
Passed!
Testing StructDefinitionError: Intervals of L and symL do not match
Passed!
Testing StructDefinitionError: p0 vanishes on [a,b]
Passed!
Test 10
Testing definition of LinearDifferentialOperator: p_k are variable Function
Passed!
Testing definition of LinearDifferentialOperator: p_k are Constants
Passed!
Testing definition of LinearDifferentialOperator: p_k are mixed
Passed!
Testing StructDefinitionError: p_k should be Function or Number
Passed!
Testing StructDefinitionError: Number of p_k and symP_k do not match
Passed!
Testing StructDefinitionError: Intervals of L and symL do not match
Passed!
Testing StructDefinitionError: p0 vanishes on [a,b]
Passed!
Test 10
Testing definition of Line

Passed!
Testing definition of LinearDifferentialOperator: p_k are Constants
Passed!
Testing definition of LinearDifferentialOperator: p_k are mixed
Passed!
Testing StructDefinitionError: p_k should be Function or Number
Passed!
Testing StructDefinitionError: Number of p_k and symP_k do not match
Passed!
Testing StructDefinitionError: Intervals of L and symL do not match
Passed!
Testing StructDefinitionError: p0 vanishes on [a,b]
Passed!
Test 10
Testing definition of LinearDifferentialOperator: p_k are variable Function
Passed!
Testing definition of LinearDifferentialOperator: p_k are Constants
Passed!
Testing definition of LinearDifferentialOperator: p_k are mixed
Passed!
Testing StructDefinitionError: p_k should be Function or Number
Passed!
Testing StructDefinitionError: Number of p_k and symP_k do not match
Passed!
Testing StructDefinitionError: Intervals of L and symL do not match
Passed!
Testing StructDefinitionError: p0 vanishes on [a,b]
Passed!
Test 10
Testing definition of Line

Passed!
Testing definition of LinearDifferentialOperator: p_k are Constants
Passed!
Testing definition of LinearDifferentialOperator: p_k are mixed
Passed!
Testing StructDefinitionError: p_k should be Function or Number
Passed!
Testing StructDefinitionError: Number of p_k and symP_k do not match
Passed!
Testing StructDefinitionError: Intervals of L and symL do not match
Passed!
Testing StructDefinitionError: p0 vanishes on [a,b]
Passed!
Test 10
Testing definition of LinearDifferentialOperator: p_k are variable Function
Passed!
Testing definition of LinearDifferentialOperator: p_k are Constants
Passed!
Testing definition of LinearDifferentialOperator: p_k are mixed
Passed!
Testing StructDefinitionError: p_k should be Function or Number
Passed!
Testing StructDefinitionError: Number of p_k and symP_k do not match
Passed!
Testing StructDefinitionError: Intervals of L and symL do not match
Passed!
Testing StructDefinitionError: p0 vanishes on [a,b]
Passed!
Test 10
Testing definition of Line

In [24]:
for n = 1:10
    println(test_vectorBoundaryFormDef(n, 10))
end

Test 1
Testing the definition of VectorBoundaryForm
Passed!
Testing StructDefinitionError: Entries of M, N should be Number
Passed!
Testing StructDefinitionError: M, N dimensions do not match
Passed!
Testing StructDefinitionError: M, N should be square matrices
Passed!
Testing StructDefinitionError: Boundary operators not linearly independent
Passed!
Test 2
Testing the definition of VectorBoundaryForm
Passed!
Testing StructDefinitionError: Entries of M, N should be Number
Passed!
Testing StructDefinitionError: M, N dimensions do not match
Passed!
Testing StructDefinitionError: M, N should be square matrices
Passed!
Testing StructDefinitionError: Boundary operators not linearly independent
Passed!
Test 3
Testing the definition of VectorBoundaryForm
Passed!
Testing StructDefinitionError: Entries of M, N should be Number
Passed!
Testing StructDefinitionError: M, N dimensions do not match
Passed!
Testing StructDefinitionError: M, N should be square matrices
Passed!
Testing StructDefinition

Passed!
Testing StructDefinitionError: M, N dimensions do not match
Passed!
Testing StructDefinitionError: M, N should be square matrices
Passed!
Testing StructDefinitionError: Boundary operators not linearly independent
Passed!
Test 3
Testing the definition of VectorBoundaryForm
Passed!
Testing StructDefinitionError: Entries of M, N should be Number
Passed!
Testing StructDefinitionError: M, N dimensions do not match
Passed!
Testing StructDefinitionError: M, N should be square matrices
Passed!
Testing StructDefinitionError: Boundary operators not linearly independent
Passed!
Test 4
Testing the definition of VectorBoundaryForm
Passed!
Testing StructDefinitionError: Entries of M, N should be Number
Passed!
Testing StructDefinitionError: M, N dimensions do not match
Passed!
Testing StructDefinitionError: M, N should be square matrices
Passed!
Testing StructDefinitionError: Boundary operators not linearly independent
Passed!
Test 5
Testing the definition of VectorBoundaryForm
Passed!
Testi

Testing StructDefinitionError: M, N should be square matrices
Passed!
Testing StructDefinitionError: Boundary operators not linearly independent
Passed!
Test 10
Testing the definition of VectorBoundaryForm
Passed!
Testing StructDefinitionError: Entries of M, N should be Number
Passed!
Testing StructDefinitionError: M, N dimensions do not match
Passed!
Testing StructDefinitionError: M, N should be square matrices
Passed!
Testing StructDefinitionError: Boundary operators not linearly independent
Passed!
true
Test 1
Testing the definition of VectorBoundaryForm
Passed!
Testing StructDefinitionError: Entries of M, N should be Number
Passed!
Testing StructDefinitionError: M, N dimensions do not match
Passed!
Testing StructDefinitionError: M, N should be square matrices
Passed!
Testing StructDefinitionError: Boundary operators not linearly independent
Passed!
Test 2
Testing the definition of VectorBoundaryForm
Passed!
Testing StructDefinitionError: Entries of M, N should be Number
Passed!
Tes